In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
import numpy as np
import datetime
import pickle

device = "cuda" if torch.cuda.is_available() else "cpu"
print(torch.cuda.get_device_name(), "|", torch.cuda.is_available())

Tesla T4 | True


In [3]:
charmap = pickle.load(open("Checkpoints/rockyou_charmap.pickle", "rb"))
inv_charmap = pickle.load(open("Checkpoints/rockyou_inv_charmap.pickle", "rb"))

# Model Definition

In [8]:
class ResidualBlock(nn.Module):
    """
    Residual blocks take BATCH_SIZE x CHANNELS x LENGTH -> BATCH_SIZE x CHANNELS x LENGTH
    """
    def __init__(self, n_channels, kernel_size=3):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=n_channels, out_channels=n_channels, kernel_size=kernel_size, padding=kernel_size // 2)
        self.conv2 = nn.Conv1d(in_channels=n_channels, out_channels=n_channels, kernel_size=kernel_size, padding=kernel_size // 2)
        
    def forward(self, inputs):
        x = F.relu(inputs)
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        outputs = inputs + x * 0.3
        return outputs
    
class Generator(nn.Module):
    def __init__(self, charmap, kernel_size=3):
        super(Generator, self).__init__()
        self.lin = nn.Linear(in_features=128, out_features=128*10) #Channels x Length
        self.block1 = ResidualBlock(128)
        self.block2 = ResidualBlock(128)
        self.block3 = ResidualBlock(128)
        self.block4 = ResidualBlock(128)
        self.block5 = ResidualBlock(128)
        self.conv = nn.Conv1d(in_channels=128, out_channels=len(charmap), kernel_size=kernel_size, padding=kernel_size//2)
        
    
    def forward(self, inputs):
        x = self.lin(inputs).reshape(-1, 128, 10) # for residual blocks
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        x = self.conv(x).permute(0, 2, 1)
        x = F.softmax(x, dim=2)
        return x
    
class Discriminator(nn.Module):
    def __init__(self, charmap, kernel_size=3):
        super(Discriminator, self).__init__()
        self.length_charmap = len(charmap)
        self.conv1 = nn.Conv1d(self.length_charmap, 128, kernel_size=kernel_size, padding=kernel_size // 2)
        self.block1 = ResidualBlock(128)
        self.block2 = ResidualBlock(128)
        self.block3 = ResidualBlock(128)
        self.block4 = ResidualBlock(128)
        self.block5 = ResidualBlock(128)
        self.flatten = nn.Flatten()
        self.lin = nn.Linear(in_features=128 * 10, out_features=1)
    
    def forward(self, inputs): #one-hot is input to the discriminator
#        x = F.one_hot(inputs, num_classes=self.length_charmap).permute(0, 2, 1).float()
        x = inputs.permute(0, 2, 1)
        x = self.conv1(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        x = self.flatten(x)
        outputs = self.lin(x)
        return outputs

In [9]:
netG = Generator(charmap).to(device)
netG.load_state_dict(torch.load("/home/nvijayakumar/gcp-gan/Checkpoints/netG900005:35:43AM_12-03-20"))

<All keys matched successfully>

In [11]:
def translate(passwords):
    return ["".join([inv_charmap[c] for c in password]) for password in passwords]

In [13]:
latent_noise = torch.randn(100, 128).to(device=device)
generated_passwords = netG(latent_noise).argmax(dim=2)
translate(generated_passwords)

['mamink603|',
 'x1Кannan||',
 's5it|ľ*|||',
 'awisalynee',
 'ayimt႑aๆ||',
 'tistnitnee',
 '5508755591',
 'Mๆ618405||',
 'anistven||',
 '400988||||',
 'nekkivene|',
 'innane||||',
 'wikI620|||',
 'ti႑anbe1||',
 'G801454|||',
 'smonkove0|',
 'thatba19||',
 'inna16896|',
 'taktovaT||',
 'wisteგe;91',
 'nitka16|||',
 'natnikx|||',
 '58693T||||',
 'snat႑mๆ|||',
 'fd443i5it|',
 '#5053|||||',
 '1034nist||',
 'inw2416865',
 'jonisji53|',
 'Y1222386||',
 'wiknitج1||',
 '1044402098',
 'slla39||||',
 'nekmam||||',
 'setea16|||',
 'snatiey|||',
 'cenkama099',
 'wikent|1||',
 'felonE1|||',
 'te1084||||',
 '65inalyne|',
 '@50835||||',
 'wiktnne|||',
 'nantan1|||',
 'katnisteta',
 'jojHes5st|',
 'eosane||||',
 '159995820|',
 'nგateiydeI',
 'mavane1КE|',
 'wimeضaתΤ||',
 'eanarlr02|',
 'tnsniwic||',
 'fnoe1280||',
 'tisanrike|',
 'nგestama||',
 'tnrets803|',
 '55933|||||',
 'niitke||||',
 'wikane2|||',
 '6354405КE|',
 'imaniiteto',
 'nictast|||',
 '13405v1|||',
 'wikeitanar',
 'sodyrict||',
 '65531504